In [56]:
import numpy as np
import pandas as pd

## sfen記法
- 先手なら大文字、後手なら小文字
| 駒 | アルファベット |
| :----: | :----: |
| 玉 | K |
| 飛車 | R |
| 角 | B |
| 金 | G |
| 銀 | S |
| 桂 | N |
| 香 | L |
| 歩 | P |

ex. ```lnsgkgsnl/1r5b1/ppppppppp/9/9/7P1/PPPPPPP1P/1B5R1/LNSGKGSNL w - 2```
空白区切りで4つのブロックで構成されている
### 1つ目：局面全体
- アルファベット：上のルールで書かれた駒
- 数字：右からnマスがどのコマも置かれていない状態
- スラッシュ(/): 1段の終わり

### 2つ目：手番
- b: 先手(Black)
- w: 後手(White)

### 3つ目：持ち駒
- アルファベット：上のルールで書かれた駒
- 数字：次の駒がその数字の枚数だけ存在する
- ```-```なら双方持ち駒なし

### 4つ目：手数
- 何手目かを表記


In [3]:
sfen = "l6nl/1r2gkg2/p1nsp2p1/2pp1ps1p/1p2P2P1/P1PP5/1PS2P2P/2KGG1R2/LN5NL w BSPbp 48"

In [7]:
koma_dic = {"K":1, "R":2, "B":3, "G":4, "S":5, "N": 6, "L": 7, "P": 8,
           "k": -1, "r": -2, "b":-3, "g":-4, "s": -5, "n": -6, "l": -7, "p": -8}
mochi_id = {"R":0, "B": 1, "G": 2, "S":3, "N": 4, "L": 5, "P": 6,
           "r":7, "b": 8, "g":9, "s":10, "n":11, "l":12, "p":13}


In [68]:
def returnVec(sfen):
    block = sfen.split(" ")
    kyokumen, bw, mochi, num = block[0], block[1], block[2], block[3]
    
    # 局面
    vec1 = []
    for i in kyokumen.split("/"):
        tmp = []
        for j in i:
            if not j.isdecimal():
                tmp.append(koma_dic[j])
            else:
                tmp.extend([0]*int(j))
        vec1.append(tmp)
    
    # 手番
    vec2 = [0]*1 if bw=="b" else [1]*1
    
    # 持ち駒
    vec3 = [0] * 7 * 2
    cnt = 1
    if mochi != "-":
        for i in range(len(mochi)):
            if not mochi[i].isdecimal():
                vec3[mochi_id[mochi[i]]] = vec3[mochi_id[mochi[i]]] + cnt
                cnt = 1
            else:
                cnt = int(mochi[i])
            
    # 手数
    vec4 = [int(num)]
    return pd.DataFrame(vec1+[vec2]+[vec3]+[vec4]).fillna(0).to_numpy()
    

In [100]:
no = 194
data = pd.read_csv("./rating_data/"+str(no)+".csv")

In [117]:
data.query("年==2021").query("月==6").query("日==22")

,年,月,日,レーティング
265,2021,6,22,1733


In [135]:
def player_condition(number, date):
    date = [int(n) for n in date]
    data = pd.read_csv("./rating_data/"+str(number)+".csv")
    oneyear = data.query("年=="+str(date[0]))
    max = oneyear["レーティング"].max()
    min = oneyear["レーティング"].min()
    playdate = data.query("年=="+str(date[0])).query("月=="+str(date[1])).query("日=="+str(date[2]))["レーティング"]
    normalize = (playdate-min) / (max-min)
    return normalize.to_numpy()

In [142]:
piyo = player_condition(194, ["2021", "06", "22"])
hoge = returnVec(sfen)
piyo.shape
hoge.shape

(12, 14)

In [144]:
huga = np.vstack([hoge, np.append(piyo, np.zeros(hoge.shape[1]-piyo.shape[0]))])

In [150]:
pd.DataFrame(huga, dtype="f8")

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,-7.000000,0.0,0.0,0.0,0.0,0.0,0.0,-6.0,-7.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,-2.0,0.0,0.0,-4.0,-1.0,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-8.000000,0.0,-6.0,-5.0,-8.0,0.0,0.0,-8.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,-8.0,-8.0,0.0,-8.0,-5.0,0.0,-8.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,-8.0,0.0,0.0,8.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0
5,8.000000,0.0,8.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.000000,8.0,5.0,0.0,0.0,8.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0
7,0.000000,0.0,1.0,4.0,4.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,7.000000,6.0,0.0,0.0,0.0,0.0,0.0,6.0,7.0,0.0,0.0,0.0,0.0,0.0
9,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
